<a href="https://colab.research.google.com/github/ParthG60/Traders_at_Berkley/blob/main/money.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Python ≥3.5 is required
#import sys #
# Scikit-Learn ≥0.20 is required
import sklearn # general ml package

# Common imports
import numpy as np # fundamental package for scientific computing
import os # to run file I/O operation 

# to make this notebook's output stable across runs
# any number will do, as long as it is used consistently
np.random.seed(42)
# To plot pretty figures

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
import pandas as pd
from google.colab import files
# uploaded = files.upload()
# import io
df2 = pd.read_csv("train.csv")

In [ ]:
test = pd.read_csv("test.csv")
train = df2

train = train.drop(["counterparty (naive)", "sm sentiment (naive)", "pnl"], axis=1)
display(train)
display(test)

,trade id,pred edge,stock,location,price,size,dir,counterparty,sm sentiment,realized edge
0,0,18.145986,B,CSE,98.507314,36.0,S,HF,2.144241,-4.007551
1,1,14.414018,C,CSE,14.935761,1094.0,S,HF,-94.202156,-3.716749
2,2,12.306395,B,NYSE,99.818500,7.0,S,R,27.572105,11.867307
3,3,0.553884,B,CSE,100.459287,59.0,B,HF,-45.020574,-16.951884
4,4,5.020094,C,NASDAQ,15.209969,449.0,S,MF,22.575639,-2.337918
...,...,...,...,...,...,...,...,...,...,...
199995,199995,1.035089,B,NYSE,98.290999,148.0,S,R,47.282761,-4.003388
199996,199996,3.668662,C,NASDAQ,15.003929,327.0,S,MF,-59.582576,0.522307
199997,199997,8.910650,D,NYSE,2020.034449,2.0,S,R,96.035708,-25.712969
199998,199998,4.287823,B,NYSE,99.224065,127.0,S,R,-90.732796,3.013510


,trade id,pred edge,stock,location,price,size,dir,counterparty (naive),sm sentiment (naive)
0,0,11.785579,C,NYSE,14.727382,1346.0,B,MF,9.119066
1,1,8.490506,C,CSE,15.016002,882.0,S,MF,-48.652384
2,2,0.687709,C,NYSE,15.169420,423.0,B,MF,3.536848
3,3,7.352720,D,CSE,2026.572586,8.0,B,MF,48.257293
4,4,2.995660,C,NYSE,15.027723,523.0,B,MF,-66.590908
...,...,...,...,...,...,...,...,...,...
99995,99995,18.867160,D,NYSE,1995.388892,1.0,B,HF,-85.018229
99996,99996,11.175501,B,NASDAQ,99.420082,51.0,S,MF,43.210479
99997,99997,2.351071,A,NASDAQ,40.410974,270.0,S,HF,29.156088
99998,99998,0.043775,C,NYSE,15.090556,787.0,B,MF,-91.163595


In [ ]:
train.info()
train.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   trade id       200000 non-null  int64  
 1   pred edge      200000 non-null  float64
 2   stock          200000 non-null  object 
 3   location       200000 non-null  object 
 4   price          200000 non-null  float64
 5   size           200000 non-null  float64
 6   dir            200000 non-null  object 
 7   counterparty   200000 non-null  object 
 8   sm sentiment   200000 non-null  float64
 9   realized edge  200000 non-null  float64
dtypes: float64(5), int64(1), object(4)
memory usage: 15.3+ MB


,trade id,pred edge,price,size,sm sentiment,realized edge
count,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000
mean,99999.500000,7.983853,343.769013,411.894570,-0.188683,0.964738
std,57735.171256,6.027436,697.452348,491.067974,57.705525,9.519775
min,0.000000,0.000091,14.364434,0.000000,-99.998373,-57.767760
25%,49999.750000,3.190248,15.085134,65.000000,-50.127629,-4.138950
50%,99999.500000,6.749716,40.267344,209.000000,-0.291352,1.302865
75%,149999.250000,11.516979,100.426625,587.000000,49.745824,6.810894
max,199999.000000,46.539412,2075.983762,3481.000000,99.998372,41.449102


In [ ]:
from sklearn.model_selection import train_test_split
# Split the data into train set (80%) and test set (20%)
train_set, test_set = train_test_split(train, test_size=0.2, random_state=42)
train_set_c, test_set_c = train_test_split(train, test_size=0.2, random_state=42)
train_set_s, test_set_s = train_test_split(train, test_size=0.2, random_state=42)






In [ ]:
train_set2 = train_set.drop(["realized edge", "price", "size", "sm sentiment", "dir"], axis=1)
train_set2_label = train_set["realized edge"].copy()

train_set2_c = train_set.drop(["realized edge","price", "size", "sm sentiment","counterparty"], axis=1)
train_set2_c_label = train_set["realized edge"].copy()





In [ ]:
sample_incomplete_rows = train_set[train_set.isnull().any(axis=1)].head()
sample_incomplete_rows

train_set.head(2)

,trade id,pred edge,stock,location,price,size,dir,counterparty,sm sentiment,realized edge
153248,153248,7.171204,C,NYSE,14.858342,1545.0,B,R,61.676452,11.314348
67802,67802,1.151436,C,CSE,14.792696,1110.0,S,MF,61.129259,-7.268935


In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline


from sklearn.preprocessing import StandardScaler
num_pipeline = Pipeline([
        ('std_scaler', StandardScaler()),
    ])

num_attribs = ["pred edge","trade id"]
cat_attribs = ["stock", "location", "counterparty"]

full_pipeline = ColumnTransformer([
                                    ("num", num_pipeline, num_attribs),
        ("cat", OneHotEncoder(), cat_attribs),
    ])

train_prepared = full_pipeline.fit_transform(train_set2)

num_attribs_c = ["pred edge","trade id"]
cat_attribs_c = ["stock", "location", "dir"]

full_pipeline_c = ColumnTransformer([
                                    ("num", num_pipeline, num_attribs_c),
        ("cat", OneHotEncoder(), cat_attribs_c),
    ])

train_c_prepared = full_pipeline_c.fit_transform(train_set2_c)

# num_attribs = ["trade id", "pred edge", "price", "size"]
# cat_attribs = ["stock", "location", "dir", "counterparty"]

# full_pipeline = ColumnTransformer([
#                                     ("num", num_pipeline, num_attribs),
#         ("cat", OneHotEncoder(), cat_attribs),
#     ])

# train_s_prepared = full_pipeline.fit_transform(train_set2_s)



In [ ]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(n_estimators=100, random_state=42)
forest_reg.fit(train_prepared, train_set2_label)

forest_reg_c = RandomForestRegressor(n_estimators=100, random_state=42)
forest_reg_c.fit(train_c_prepared, train_set2_c_label)

RandomForestRegressor(random_state=42)

In [ ]:
from sklearn.metrics import mean_squared_error

train_predictions = forest_reg.predict(train_prepared)
forest_mse = mean_squared_error(train_set2_label, train_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

train_c_predictions = forest_reg_c.predict(train_c_prepared)
forest_c_mse = mean_squared_error(train_set2_c_label, train_c_predictions)
forest_rmse_c = np.sqrt(forest_c_mse)
forest_rmse_c


2.9624533334047665

In [ ]:
print(forest_rmse, forest_rmse_c)

2.7088380112334796 2.9624533334047665


In [ ]:
train_predictions
print((train_predictions > 3).sum())
print((train_c_predictions > 3 ).sum())
train_predictions

61526
59529


array([ 8.71569819, -6.7419367 ,  0.98056765, ..., -0.95151568,
       17.30118401, -2.823409  ])

In [ ]:
price = test[["price"]].to_numpy()
size = test[["size"]].to_numpy()

In [ ]:

test_withc = test.drop(["sm sentiment (naive)", "price","size", "dir"], axis=1)
test_without_c = test.drop(["sm sentiment (naive)", "counterparty (naive)", "price","size"], axis=1)

# final_model = lin_reg


# y_test = test_set["realized edge"].copy()


# X_test_prepared = full_pipeline.transform(test_set)

# print(X_test_prepared[0])

# final_predictions = final_model.predict(X_test_prepared)


# final_mse = mean_squared_error(y_test, final_predictions)
# final_rmse = np.sqrt(final_mse)

# final_rmse



In [ ]:
test_withc
train_prepared

array([[-0.1338299 ,  0.9231032 ,  0.        , ...,  0.        ,
         0.        ,  1.        ],
       [-1.1323603 , -0.55707767,  0.        , ...,  0.        ,
         1.        ,  0.        ],
       [-0.73146961,  0.84759226,  0.        , ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [-1.0505307 ,  0.55384618,  1.        , ...,  0.        ,
         0.        ,  1.        ],
       [ 2.84302518,  0.81256516,  0.        , ...,  0.        ,
         1.        ,  0.        ],
       [-1.29062144,  0.3810666 ,  0.        , ...,  0.        ,
         0.        ,  1.        ]])

In [ ]:
# final_model_c = forest_reg_c

# X_test_prepared = full_pipeline.transform(test)


# final_predictions_c= final_model_c.predict(X_test_prepared)


# print(final_predictions_c.size)


final_model_withc = forest_reg_c


X_test_prepared = full_pipeline_c.transform(test_without_c)

final_predictions_withoutc = final_model_withc.predict(X_test_prepared)


print(final_predictions_withoutc)



# final_mse = mean_squared_error(y_test, final_predictions)
# final_rmse = np.sqrt(final_mse)

# final_rmse



[  6.14970005  -1.23924344 -10.43696072 ...   1.60801888  -3.33986852
   1.77161302]


In [ ]:
test_withc

,trade id,pred edge,stock,location,counterparty (naive)
0,0,11.785579,C,NYSE,MF
1,1,8.490506,C,CSE,MF
2,2,0.687709,C,NYSE,MF
3,3,7.352720,D,CSE,MF
4,4,2.995660,C,NYSE,MF
...,...,...,...,...,...
99995,99995,18.867160,D,NYSE,HF
99996,99996,11.175501,B,NASDAQ,MF
99997,99997,2.351071,A,NASDAQ,HF
99998,99998,0.043775,C,NYSE,MF


In [ ]:
final_model = forest_reg

test_withc = test_withc.rename(columns={"counterparty (naive)": "counterparty"})
print(test_withc)
print(train_set2)

X_test_prepared = full_pipeline.transform(test_withc)

final_predictions = final_model.predict(X_test_prepared)
print(final_predictions)

       trade id  pred edge stock location counterparty
0             0  11.785579     C     NYSE           MF
1             1   8.490506     C      CSE           MF
2             2   0.687709     C     NYSE           MF
3             3   7.352720     D      CSE           MF
4             4   2.995660     C     NYSE           MF
...         ...        ...   ...      ...          ...
99995     99995  18.867160     D     NYSE           HF
99996     99996  11.175501     B   NASDAQ           MF
99997     99997   2.351071     A   NASDAQ           HF
99998     99998   0.043775     C     NYSE           MF
99999     99999   9.204874     A     NYSE            R

[100000 rows x 5 columns]
        trade id  pred edge stock location counterparty
153248    153248   7.171204     C     NYSE            R
67802      67802   1.151436     C      CSE           MF
148889    148889   3.568257     C     NYSE            R
103093    103093   2.425595     C     NYSE           MF
104681    104681   1.938638     C

In [ ]:
arr = []
for x in range(100000):
  arr.append([x, final_predictions[x]])

dataframe = pd.DataFrame(arr) 
dataframe.columns=['trade id', 'realized edge']
dataframe
dataframe.to_csv("./hafa.csv")

In [ ]:
print(final_predictions_withoutc)
# print(final_predictions_wc)


[  6.14970005  -1.23924344 -10.43696072 ...   1.60801888  -3.33986852
   1.77161302]


In [ ]:
final_withc = []
final_withoutc = []

final_withc_above3 = []
final_withoutc_above3 = []

for x in range(100000):
  if final_predictions[x] > 2.5:
    final_withc_above3.append(x)
  if final_predictions_withoutc[x] > 3:
    final_withoutc_above3.append(x)

for x in range(100000):
  final_withc.append(price[x][0] * size[x][0] * final_predictions[x]/10000)
  final_withoutc.append( price[x][0] * size[x][0] * final_predictions[x]/10000)
print( final_withoutc)


[11.422730840550681, 2.0182756956764227, -1.677303416497925, -2.2183660518906194, -2.518207732475723, -8.377318070899202, 6.781955581121544, 0.6908846629992047, 4.7773418347593575, -5.194122363451539, -4.0657301054130475, -1.307550464723921, 15.360607080237248, -0.5639120630025047, 4.223726632829288, 1.712843530664192, 3.5396830173752134, -22.849566803083576, -2.455098701826382, 7.692159612861529, -16.589182963204465, -2.7829731906416266, 19.704859966420525, 0.46870371040163156, -0.11702719669310584, -21.595720301097952, 13.243647825991196, 4.215187757390173, -1.2816285112988184, 4.892570552729017, -28.594502074304927, 3.5764734886574026, -4.42618275776574, 1.8514430135996183, 7.3075896209110365, -13.920280590681863, 1.6644656321329085, 2.0938241759352025, 1.7903793021937269, -35.83654051677176, 4.936635461310787, -8.47428531799507, -1.1787825839071064, -2.385166840451186, 21.564871419101696, 6.261906613719173, 3.4362549263758906, 1.216428274124981, 4.228967169053655, -6.39054625191892

In [ ]:
print(final_withc_above3)
print(final_withoutc_above3)

[0, 6, 12, 14, 19, 22, 26, 27, 31, 34, 37, 40, 44, 45, 46, 54, 59, 64, 65, 68, 70, 80, 82, 83, 85, 90, 94, 98, 104, 109, 112, 113, 115, 122, 126, 129, 132, 134, 135, 137, 138, 146, 148, 149, 151, 155, 156, 174, 176, 179, 181, 184, 189, 191, 195, 202, 210, 211, 213, 216, 219, 222, 223, 225, 227, 229, 231, 232, 233, 238, 243, 245, 246, 247, 249, 252, 254, 255, 258, 260, 262, 265, 266, 267, 271, 272, 273, 278, 281, 283, 291, 292, 296, 297, 304, 307, 308, 309, 317, 318, 319, 323, 328, 330, 336, 337, 338, 353, 360, 361, 362, 368, 371, 381, 384, 386, 387, 388, 391, 392, 393, 395, 397, 400, 401, 403, 407, 410, 414, 419, 420, 421, 422, 425, 428, 430, 436, 437, 438, 441, 449, 451, 456, 457, 468, 470, 471, 472, 477, 486, 493, 494, 497, 498, 505, 506, 511, 513, 515, 516, 521, 522, 525, 526, 528, 533, 534, 535, 537, 547, 549, 550, 553, 554, 555, 556, 559, 560, 562, 563, 565, 567, 573, 576, 582, 583, 584, 587, 593, 594, 597, 602, 603, 605, 606, 616, 617, 619, 622, 625, 626, 627, 628, 629, 639, 640,

In [ ]:
profit_withc = 0
for x in final_withc_above3:
  profit_withc += final_withc[x]

profit_withoutc = 0
for x in final_withoutc_above3:
  profit_withoutc += final_withoutc[x]

# print(profit_withc)
print(profit_withoutc, profit_withc)

323015.6895723815 396659.25212887407


In [ ]:
big = []

for x in range(100000):
  if size[x] * price[x] > 18107:
    big.append(1)
  else:
    big.append(0)

big


[1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,


In [ ]:

n = []
for x in range(100000):
  if final_predictions[x] > 5:
    n.append(1)
  elif big[x] == 1 and final_predictions[x] > 2.5:
    n.append(1)
  else :
    n.append(0)

print(n)


[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 

In [ ]:
# from sklearn.metrics import mean_absolute_error

# lin_mae = mean_absolute_error(y_test, final_predictions)
# lin_mae
arr = []
for x in range(100000):
  arr.append([x, n[x]])

dataframe = pd.DataFrame(arr) 
dataframe.columns=['trade id', 'trade']
dataframe
dataframe.to_csv("./newStrat4.csv")